In [ ]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1Mv7vKoI-QL6kV-1TIDE7N67_L0LXvJAg
!unzip /content/ANDL2.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import os
import shutil
from datetime import datetime
from functools import partial

from PIL import Image

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorboard import program

SEED = 1996

In [ ]:
# Batch size
bs = 32

# img shape
img_h = 256
img_w = 256

num_classes=3
# Load Xception Model
Xception = tf.keras.applications.Xception(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

83689472/83683744 [==============================] - 0s 0us/step


In [ ]:
Xception.summary()

In [ ]:
train_path = os.path.join(os.getcwd(), 'MaskDataset/training')
test_path  = os.path.join(os.getcwd(), 'MaskDataset/test')

In [ ]:
def divideDatasetInTargetFolders(json_definition, dataset_path):
    for elem in json_definition:
        dest_dir = os.path.join(dataset_path, str(json_definition[elem]))
        if not os.path.isdir(dest_dir):
            os.mkdir(dest_dir)
        try:
            shutil.move(os.path.join(dataset_path, elem),
                        os.path.join(dest_dir, elem)
                        )
        except FileNotFoundError as e:
            print("File not found: " + str(e))
            continue
    os.mkdir(os.path.join(dataset_path, "augmented"))
    os.mkdir(os.path.join(dataset_path, "augmented/training"))
    os.mkdir(os.path.join(dataset_path, "augmented/validation"))

In [ ]:
division_dict = json.load(
  open(os.path.join(os.getcwd(), 'MaskDataset/train_gt.json'))
)

divideDatasetInTargetFolders(division_dict, train_path)

In [ ]:
preproc_fun_fixed = partial(tf.keras.preprocessing.image.smart_resize, size=(img_w, img_h))

train_data_gen = ImageDataGenerator(rotation_range=10,
                                    width_shift_range=10,
                                    height_shift_range=10,
                                    zoom_range=0.3,
                                    horizontal_flip=True,
                                    fill_mode='reflect',
                                    rescale=1. / 255,
                                    validation_split=0.3,
                                    preprocessing_function=preproc_fun_fixed
                                    )

test_data_gen = ImageDataGenerator(rescale=1. / 255, preprocessing_function = preproc_fun_fixed)

classes = ['0', '1', '2']
save_dir = os.path.join(train_path, 'augmented')

train_gen = train_data_gen.flow_from_directory(train_path,
                                               target_size=(img_w, img_h),
                                               seed=SEED,
                                               classes=classes,
                                               subset='training',
                                               shuffle=True,
                                               batch_size=bs
                                               )

valid_gen = train_data_gen.flow_from_directory(train_path,
                                               target_size=(img_w, img_h),
                                               seed=SEED,
                                               classes=classes,
                                               subset='validation',
                                               shuffle=False,
                                               batch_size=bs
                                             )
#new
import pandas as pd
images = [f for f in os.listdir(test_path)]
images = pd.DataFrame(images)
images.rename(columns = {0:'filename'}, inplace = True)
images["class"] = 'test'

test_gen = test_data_gen.flow_from_dataframe(images,
                                               test_path,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)



#end new


train_set = tf.data.Dataset.from_generator(lambda: train_gen,
                                           output_types=(tf.float32, tf.float32),
                                           output_shapes=(
                                               [None, img_w, img_h, 3],
                                               [None, len(classes)]
                                           ))

validation_set = tf.data.Dataset.from_generator(lambda: valid_gen,
                                                output_types=(tf.float32, tf.float32),
                                                output_shapes=(
                                                    [None, img_w, img_h, 3],
                                                    [None, len(classes)]
                                                ))

test_set = tf.data.Dataset.from_generator(lambda: test_gen,
                                          output_types=(tf.float32, tf.float32),
                                          output_shapes=(
                                              [None, img_w, img_h, 3],
                                              [None, len(classes)]
                                          ))

train_set.repeat()
validation_set.repeat()
test_set.repeat()

In [ ]:
callbacks = []
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights= True)
    callbacks.append(es_callback)
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = os.getcwd() + '/drive/My Drive/weights_xc.h5',
      verbose=1, save_best_only=True, save_weights_only=True)
    callbacks.append(cp_callback)

In [14]:
# Create Model
# ------------

finetuning = True

if finetuning:
    freeze_until = 1 # layer from which we want to fine-tune
    
    for layer in Xception.layers[:freeze_until]:
        layer.trainable = False
else:
    Xception.trainable = False
    
model = tf.keras.Sequential()
model.add(Xception)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(units=512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(units=256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

# Visualize created model as a table
model.summary()

# Visualize initialized weights
model.weights
loss = tf.keras.losses.CategoricalCrossentropy()
# maybe explore learning rate solutions
lr = 1e-6
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
metrics = ['accuracy']
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)


train = False
retrain = False
if train:
  if retrain:
    model.load_weights('/content/drive/My Drive/weights_xc.h5')
  model.fit(x=train_set,
            epochs=30,  #### set repeat in training dataset
            steps_per_epoch=len(train_gen),
            validation_data=validation_set,
            validation_steps=len(valid_gen),
            callbacks=callbacks)
else:
  model.load_weights('/content/drive/My Drive/weights_xc.h5')

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 8, 8, 2048)        20861480  
_________________________________________________________________
flatten_3 (Flatten)          (None, 131072)            0         
_________________________________________________________________
dense_12 (Dense)             (None, 1024)              134218752 
_________________________________________________________________
dropout_9 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 256)              

In [15]:
def create_csv(results, results_dir='/content/drive/My Drive'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [16]:
test_gen.reset()
predictions = model.predict_generator(test_gen, len(test_gen), verbose=1)

Instructions for updating:
Please use Model.predict, which supports generators.
15/15 [==============================] - 7s 461ms/step


In [17]:
import numpy as np

results = {}
images = test_gen.filenames
i = 0

for p in predictions:
  prediction = np.argmax(p)
  import ntpath
  image_name = ntpath.basename(images[i])
  results[image_name] = str(prediction)
  i = i + 1

In [18]:
create_csv(results)

In [19]:
# Save the model
save = True
if save:
    model.save('/content/drive/My Drive/model_xc.h5')